In [ ]:
#git clone https://github.com/lucidrains/iTransformer.git

In [1]:
# TODO which one?
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer
import torch
from pathlib import Path

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from utils import data_handling, helpers
import config

window_size = 96
pred_length = (96, 192, 336, 720)

print("Import succesfull")


Import succesfull


In [19]:
# eu electricity data
data_tensor = data_handling.eu_electricity_to_tensor()
data_dict, standardize_values =  data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to dataloader
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 208])


In [13]:
# use electricity dataset
data_dict = data_handling.load_electricity()

# train for eu_electricity
for key, values in data_dict.items():
    data_dict[key] = values[:, 0:208]

dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 208])


In [14]:
# for electricity
best_parameters = {'depth': 3, 'dim': 818, 'dim_head': 58, 'heads': 4, 'attn_dropout': 0.17358586508257956, 'ff_mult': 4, 'ff_dropout': 0.07147101457460232, 'num_mem_tokens': 4, 'learning_rate': 0.0008061714887745944}

model_config = {
    'num_variates': data_dict["train"].size(1),
    'lookback_len': window_size,
    'depth': best_parameters["depth"],
    'dim': best_parameters["dim"],
    'num_tokens_per_variate': 1,
    'pred_length': pred_length,
    'dim_head': best_parameters["dim_head"],
    'heads': best_parameters["heads"],
    'attn_dropout': best_parameters["attn_dropout"],
    'ff_mult': best_parameters["ff_mult"],
    'ff_dropout': best_parameters["ff_dropout"],
    'num_mem_tokens': best_parameters["num_mem_tokens"],
    'use_reversible_instance_norm': True,
    'reversible_instance_norm_affine': False,
    'flash_attn': True
}

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# defining all needed instances
model = iTransformer(**model_config).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["revin"])


Using device: cuda


In [16]:
def fast_eval(dataloader):
	model.eval()
	preds_dict = {
		96 : {
				"mse" : 0, "mae" : 0, "smape":0, "p10" : 0, "p50" : 0, "p90" : 0
		},
		192 : {
				"mse" : 0, "mae" : 0, "smape":0, "p10" : 0, "p50" : 0, "p90" : 0
		},
		336 : {
				"mse" : 0, "mae" : 0, "smape":0, "p10" : 0, "p50" : 0, "p90" : 0
		},
		720: {
				"mse" : 0, "mae" : 0, "smape":0, "p10" : 0, "p50" : 0, "p90" : 0
		}}

	with torch.no_grad():
		for input, target in tqdm(dataloader, desc=f"Epoch: Validating"):
		
			# targets are saved as list, send each to device
			targets = (target[0].to(device), target[1].to(device), target[2].to(device), target[3].to(device))
			outputs = model(input.to(device))
		
		# for each prediciton length we calculate the metrics
		for target, output in zip(targets, outputs.values()):
			preds_dict[output.size(1)]["mse"] = preds_dict[output.size(1)]["mse"] + helpers.mean_squared_error(output, target)
			preds_dict[output.size(1)]["mae"] = preds_dict[output.size(1)]["mae"] + helpers.mean_absolute_error(output, target)
			preds_dict[output.size(1)]["smape"] = preds_dict[output.size(1)]["smape"] + helpers.symmetric_mean_absolute_percentage_error(output, target)
			preds_dict[output.size(1)]["p10"] = preds_dict[output.size(1)]["p10"] + helpers.percentile(output, target, 0.1)
			preds_dict[output.size(1)]["p50"] = preds_dict[output.size(1)]["p50"] + helpers.percentile(output, target, 0.5)
			preds_dict[output.size(1)]["p90"] = preds_dict[output.size(1)]["p90"] + helpers.percentile(output, target, 0.9)

	#print("Pred_len 96 MSE: ", preds_dict[96]["mse"])
	
	return preds_dict

In [22]:
def train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer):
	global_step = 0
	model.train()
	total_loss = 0
	
	for input, target in tqdm(dataloader_train, desc=f"Epoch: {epoch}"):
		optimizer.zero_grad()
		if len(target) == 4:
			loss = model(input.to(device), (target[0].to(device), target[1].to(device), target[2].to(device), target[3].to(device)))
		else:
			loss = model(input.to(device), target.to(device))
		
		loss.backward()
		optimizer.step()
		total_loss += loss.item()

		writer.add_scalar('train_loss', loss, global_step)
		writer.add_scalar('train_lr', optimizer.param_groups[0]['lr'], global_step)
		lr =  optimizer.param_groups[0]['lr']
		global_step+=1

	print(f'Epoch {epoch}, MSE-Loss: {total_loss / (len(dataloader_train) * 4)}, LR: {lr}')

	scheduler.step()
	writer.close()
	if epoch % 5 == 0:
		helpers.create_checkpoint(model, optimizer, scheduler, epoch, loss, global_step, "iTransfomrer_electricity")
	eval_metrics_dict = fast_eval(dataloader_validation)

epoch = 1

for epoch in range(1, epoch + 1):
	train_one_epoch(epoch, model, device, dataloader_validation, dataloader_validation, optimizer, scheduler, writer)


Epoch: 1: 100%|██████████| 111/111 [00:11<00:00,  9.30it/s]


Epoch 1, MSE-Loss: 0.7487929769464441, LR: 8.061714887745946e-07


Epoch: Validating: 100%|██████████| 111/111 [00:03<00:00, 29.65it/s]

Pred_len 96 MSE:  tensor(0.7798, device='cuda:0')


In [7]:
metrics = fast_eval(dataloader_test)
metrics

Epoch: Validating: 100%|██████████| 67/67 [00:03<00:00, 17.15it/s]

dict_items([('mse', tensor(0.1776, device='cuda:0')), ('mae', tensor(0.2737, device='cuda:0')), ('smape', tensor(112.4903, device='cuda:0')), ('p10', tensor(0.0357, device='cuda:0')), ('p50', tensor(0.1899, device='cuda:0')), ('p90', tensor(0.5723, device='cuda:0'))])


{96: {'mse': tensor(0.1501, device='cuda:0'),
  'mae': tensor(0.2374, device='cuda:0'),
  'smape': tensor(101.7728, device='cuda:0'),
  'p10': tensor(0.0307, device='cuda:0'),
  'p50': tensor(0.1633, device='cuda:0'),
  'p90': tensor(0.4853, device='cuda:0')},
 192: {'mse': tensor(0.1776, device='cuda:0'),
  'mae': tensor(0.2737, device='cuda:0'),
  'smape': tensor(112.4903, device='cuda:0'),
  'p10': tensor(0.0357, device='cuda:0'),
  'p50': tensor(0.1899, device='cuda:0'),
  'p90': tensor(0.5723, device='cuda:0')},
 336: {'mse': tensor(0.1840, device='cuda:0'),
  'mae': tensor(0.2865, device='cuda:0'),
  'smape': tensor(114.6408, device='cuda:0'),
  'p10': tensor(0.0380, device='cuda:0'),
  'p50': tensor(0.1987, device='cuda:0'),
  'p90': tensor(0.6163, device='cuda:0')},
 720: {'mse': tensor(0.3362, device='cuda:0'),
  'mae': tensor(0.3728, device='cuda:0'),
  'smape': tensor(132.2587, device='cuda:0'),
  'p10': tensor(0.0414, device='cuda:0'),
  'p50': tensor(0.2288, device='cuda:0

In [23]:
# predict on eu electricity
metrics = fast_eval(dataloader_test)
metrics

Epoch: Validating: 100%|██████████| 248/248 [00:08<00:00, 28.32it/s]

Pred_len 96 MSE:  tensor(1.0267, device='cuda:0')


{96: {'mse': tensor(1.0267, device='cuda:0'),
  'mae': tensor(0.6372, device='cuda:0'),
  'smape': tensor(157.1326, device='cuda:0'),
  'p10': tensor(0.0509, device='cuda:0'),
  'p50': tensor(0.3867, device='cuda:0'),
  'p90': tensor(1.4745, device='cuda:0')},
 192: {'mse': tensor(1.6828, device='cuda:0'),
  'mae': tensor(0.8021, device='cuda:0'),
  'smape': tensor(170.9238, device='cuda:0'),
  'p10': tensor(0.0637, device='cuda:0'),
  'p50': tensor(0.4290, device='cuda:0'),
  'p90': tensor(2.1294, device='cuda:0')},
 336: {'mse': tensor(1.9419, device='cuda:0'),
  'mae': tensor(0.8371, device='cuda:0'),
  'smape': tensor(170.4025, device='cuda:0'),
  'p10': tensor(0.0720, device='cuda:0'),
  'p50': tensor(0.4566, device='cuda:0'),
  'p90': tensor(2.1610, device='cuda:0')},
 720: {'mse': tensor(2.1341, device='cuda:0'),
  'mae': tensor(0.8491, device='cuda:0'),
  'smape': tensor(180.1998, device='cuda:0'),
  'p10': tensor(0.0843, device='cuda:0'),
  'p50': tensor(0.4829, device='cuda:0